# 머신러닝 모델링을 통한 서울시 집값 예측하기

# 1.데이터 선정 이유 및 문제 정의

In [ ]:
from IPython.display import Image
Image('/content/drive/MyDrive/aibootcamp/project2/fewfawe.png')

In [ ]:
# 매도인의 입장: 자신의 집 적절한 가격과 이윤을 남을 수 있을지 예측
# 매수인의 입장: 사고자하는 집의 적절한 가격을 예측
# 집값 예측 > 집매매 선택에 도움을 주기위해 

# 2.데이터를 이용한 가설 및 평가지표, 베이스라인 선택

In [ ]:
# 가설: 집값 예측
# 베이스라인: 평균기준모델
# 평가지표: R2

In [ ]:
!pip uninstall pandas_profiling

In [ ]:
!pip install pandas_profiling

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error
import matplotlib as mpl
import matplotlib.font_manager as fm

In [ ]:
!pip install eli5

In [ ]:
!pip install shap

In [ ]:
!pip install shap.initjs()

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# Colab 의 한글 폰트 설정
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

In [ ]:
!pip install category_encoders

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/aibootcamp/project2/서울특별시_부동산_실거래가_정보_2020년.csv',encoding ='cp949')
df.head()

#3.EDA와 데이터 전처리

## • EDA

> 1. 중복 혹은 필요없는 컬럼 존재.
>> * 필요없는 컬럼: '실거래가아이디','신고년도','업무구분코드','업무구분','물건번호', '건물명', '관리구분코드','지번코드'
>> * 중복: '시군구코드','법정동코드''건물주용도코드'
>> *        -> 자치구명   -> 법정동명 -> 건물주용도
> 2. Target값에 이상치 존재.

In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(df, minimal=True).to_notebook_iframe()

In [ ]:
df.describe()

## •	Feature Engineering

> 1. 중복 혹은 필요없는 컬럼 제거.
> 2. 타겟값 단위(천만원) 조정

In [ ]:
dftr = df.drop(['실거래가아이디','시군구코드','지번코드','법정동코드','신고년도','업무구분코드','업무구분','물건번호','건물명','건물주용도코드','관리구분코드'],axis=1)
dftr['물건금액'] = dftr['물건금액']/10000000
dftr

## • 이상치 제거

In [ ]:
target = '물건금액'
dftr = dftr[dftr[target] < np.percentile(dftr[target], 99.5)]

## • 결측치 제거 혹은 대체

In [ ]:
dftr.isna().sum()

> 1. 대지권면적 결측치 컬럼 제거
> 2. 층정보의 결측치는 모두 단독주택 so 평균적 2층으로 대체
> 3. if 건축년도 = 0: NAN으로 가정. 건축년도 평균값으로 대체 

In [ ]:
dftr = dftr.dropna(subset=['대지권면적'])
dftr['층정보']= dftr['층정보'].fillna(value = 2)
dftr['건축년도'] = np.where(dftr['건축년도']==0,np.nan,dftr['건축년도'])
dftr['건축년도'] = dftr['건축년도'].fillna(value = round(dftr['건축년도'].mean()))
dftr = dftr.reset_index()
dftr = dftr.iloc[:,1:]
dftr.describe()

In [ ]:
dftr.isna().sum()

In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(dftr, minimal=True).to_notebook_iframe()

#4.머신러닝 방식 적용 및 교차검증

> Train:Val:Test = 6:4:4 비율로 적용

In [ ]:
### 베이스 라인

In [ ]:
predict = dftr['물건금액'].mean()
base_y = pd.Series([predict]*len(dftr))
r2_score(dftr['물건금액'],base_y)

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(dftr, test_size = 0.2)
train, val = train_test_split(train, test_size = 0.2)

target = '물건금액'

x_train = train.drop('물건금액',axis = 1)
y_train = train['물건금액']

x_val = val.drop('물건금액',axis = 1)
y_val = val['물건금액']

x_test = test.drop('물건금액',axis = 1)
y_test = test['물건금액']

In [ ]:
from sklearn.pipeline import make_pipeline
# 인코더 라이브러리
from category_encoders import OrdinalEncoder
from category_encoders import OneHotEncoder
from category_encoders import TargetEncoder
# 대체 라이브러리
from sklearn.impute import SimpleImputer
# 스케일러 라이브러리
from sklearn.preprocessing import StandardScaler
# 모델 라이브러리
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import TransformedTargetRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV

## (1) Multiple Linear

> 다중선형회귀 결과: R2가 약 0.7 나오는 것을 확인할 수 있다.

In [ ]:
linear = make_pipeline(
    TargetEncoder(),
    SimpleImputer(),
    StandardScaler(),
    LinearRegression()
)

linear.fit(x_train, y_train)
print('R^2: ', linear.score(x_val, y_val))

In [ ]:
target = '물건금액' 
fts = dftr.columns.drop(target)
coefficients = linear.named_steps['linearregression'].coef_
pd.Series(coefficients, fts)

## (2) RidgeRegression

> 릿지회귀 결과: R2가 약 0.75 나오는 것을 확인할 수 있다.

In [ ]:
ridge = make_pipeline(
    OneHotEncoder(use_cat_names=True), 
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    Ridge()
)

ridge.fit(x_train, y_train)
print('R^2: ', ridge.score(x_val, y_val))

#### RidgeRegression hyper parameter

> 릿지선형회귀 튜닝 결과: R2가 약 0.7 나오는 것을 확인할 수 있다
> * 최적 하이퍼파라미터: alpha = 10

In [ ]:
dists = {
    'ridge__alpha': [0.1, 1, 5, 10], 
}
clf_ridge = RandomizedSearchCV(
    ridge, 
    param_distributions=dists, 
    n_iter=10, 
    cv=5,
    scoring='r2',
    verbose=1,
    n_jobs=-1
)
clf_ridge.fit(x_train, y_train);
print('최적 하이퍼파라미터: ', clf_ridge.best_params_)
print('R2: ', clf_ridge.best_score_)

## (3) RandomForest

> 랜덤포레스트 결과: R2가 약 0.84 나오는 것을 확인할 수 있다

In [ ]:
p = make_pipeline(
    OrdinalEncoder() ,
    SimpleImputer(),
    StandardScaler(),
    RandomForestRegressor(random_state=10, n_jobs=-1)
)
p.fit(x_train, y_train)
## Import up sound alert dependencies
from IPython.display import Audio, display
def allDone():
  display(Audio(url='https://freesound.org/data/previews/219/219244_4082826-lq.mp3', autoplay=True))
## Insert whatever audio file you want above
allDone()

In [ ]:
print('검증 정확도', p.score(x_val,y_val))

#### RandomForest hyper parameter

> 랜덤포레스트 튜닝 결과: R2가 약 0.84 나오는 것을 확인할 수 있다
> * 최적 하이퍼파라미터:
>> 1. max_depth: 20
>> 2. max_features: 0.82
>> 3. n_estimators: 244

In [ ]:
dists = {
    'randomforestregressor__n_estimators': randint(50, 500), 
    'randomforestregressor__max_depth': [5, 10, 15, 20, None], 
    'randomforestregressor__max_features': uniform(0, 1) # max_features
}
clf_rf = RandomizedSearchCV(
    p, 
    param_distributions=dists, 
    n_iter=10, 
    cv=5, 
    scoring='r2',  
    verbose=1,
    n_jobs=-1
)
clf_rf.fit(x_train, y_train)
print('최적 하이퍼파라미터: ', clf_rf.best_params_)
print('r2: ', clf_rf.best_score_)

## (4) BOOSTING

> 부스팅 결과: R2가 약 0.85 나오는 것을 확인할 수 있다

In [ ]:
encoder = OrdinalEncoder()
x_train_encoded = encoder.fit_transform(x_train) # 학습데이터
x_val_encoded = encoder.transform(x_val) # 검증데이터
x_test_encoded = encoder.transform(x_test) 

boosting = XGBRegressor(
    n_estimators=1000,
    objective='reg:squarederror', # default
    learning_rate=0.2,
    n_jobs=-1
)
eval_set = [(x_train_encoded, y_train), 
            (x_val_encoded, y_val)]
boosting.fit(x_train_encoded, y_train, 
          eval_set=eval_set,
          early_stopping_rounds=50
         )

In [ ]:
y_pred_boost = boosting.predict(x_val_encoded)
print('R^2: ', r2_score(y_val, y_pred_boost))
print('MAE: ', mean_absolute_error(y_val, y_pred_boost))
print('MSE: ', mean_squared_error(y_val,y_pred_boost))

#### Boosting hyper parameter

> XGB 부스팅 튜닝 결과: R2가 약 0.85 나오는 것을 확인할 수 있다
> * 최적 하이퍼파라미터:
>> 1. n_estimators: 260
>> 2. earning_rate: 0.1

In [ ]:
results = boosting.evals_result()
train_error = results['validation_0']['rmse']
val_error = results['validation_1']['rmse']
epoch = range(1, len(train_error)+1)
plt.plot(epoch, train_error, label='Train')
plt.plot(epoch, val_error, label='Validation')
plt.ylabel('rmse')
plt.xlabel('Model Complexity (n_estimators)')
plt.legend();

In [ ]:
dists = {
    'xgbregressor__n_estimators': randint(200, 300),
    'xgbregressor__learning_rate': [0.1,0.2,0.3]
}
clf_xgb = RandomizedSearchCV(
    boosting, 
    param_distributions=dists, 
    n_iter=5, 
    cv=5, 
    scoring='r2',  
    verbose=1,
    n_jobs=-1
)
clf_xgb.fit(x_train_encoded, y_train)
print('최적 하이퍼파라미터: ', clf_xgb.best_params_)
print('r2: ', clf_xgb.best_score_)

In [ ]:
y_pred1 = linear.predict(x_val)
mae1 = mean_absolute_error(y_val, y_pred1)
mse1 = mean_squared_error(y_val,y_pred1)

y_pred2 = ridge.predict(x_val)
mae2 = mean_absolute_error(y_val, y_pred2)
mse2 = mean_squared_error(y_val, y_pred2)

a3 = clf_ridge.best_estimator_
y_pred3 = a3.predict(x_val)
mae3 = mean_absolute_error(y_val, y_pred3)
mse3 = mean_squared_error(y_val, y_pred3)

y_pred4 = p.predict(x_val)
mae4 = mean_absolute_error(y_val, y_pred4)
mse4 = mean_squared_error(y_val, y_pred4)

a5 = clf_rf.best_estimator_
y_pred5 = a5.predict(x_val)
mae5 = mean_absolute_error(y_val, y_pred5)
mse5 = mean_squared_error(y_val, y_pred5)

mae6 = mean_absolute_error(y_val, y_pred_boost)
mse6 = mean_squared_error(y_val, y_pred_boost)

a7 = clf_xgb.best_estimator_
y_pred7 = a7.predict(x_val_encoded)
mae7 = mean_absolute_error(y_val, y_pred7)
mse7 = mean_squared_error(y_val, y_pred7)

## • 최종 모델 설정

In [ ]:
print('[Validation R^2 값 비교]')
print('Linear Regression: ', round(linear.score(x_val, y_val),2))
print('Ridge Regression: ', round(ridge.score(x_val, y_val),2))
print('Ridge Regression(tuning): ', round(clf_ridge.best_score_,2))
print('Random Forest: ', round(p.score(x_val,y_val),2))
print('Random Forest(tuning): ', round(clf_rf.best_score_,2))
print('Boosting: ', round(r2_score(y_val, y_pred_boost),2))
print('Boosting(tuning): ', round(clf_xgb.best_score_,2))

In [ ]:
print('[Validation MAE 값 비교]')
print('Linear Regression: ', round(mae1,2))
print('Ridge Regression: ', round(mae2,2))
print('Ridge Regression(tuning): ', round(mae3,2))
print('Random Forest: ', round(mae4,2))
print('Random Forest(tuning): ', round(mae5,2))
print('Boosting: ', round(mae6,2))
print('Boosting(tuning): ', round(mae7,2))

In [ ]:
print('[Validation MSE 값 비교]')
print('Linear Regression: ', round(mse1,2))
print('Ridge Regression: ', round(mse2,2))
print('Ridge Regression(tuning): ', round(mse3,2))
print('Random Forest: ', round(mse4,2))
print('Random Forest(tuning): ', round(mse5,2))
print('Boosting: ', round(mse6,2))
print('Boosting(tuning): ', round(mse7,2))

> 최종모델: Boosting(tuning)

> * 최적 하이퍼파라미터:
>> 1. n_estimators: 260
>> 2. earning_rate: 0.1

In [ ]:
y_pred = a7.predict(x_test_encoded)
r2_score(y_test, y_pred)

#5.머신러닝 모델 해석

In [ ]:
### (제출폼 과제) 이곳에서 과제를 진행해 주세요 ###
import shap

In [ ]:
# summary plot
shap.initjs()
shap_values = explainer.shap_values(x_train_encoded.iloc[:1000])
shap.summary_plot(shap_values, x_train_encoded, plot_type='bar')

# feature importance
importances = pd.Series(a7.feature_importances_, x_train.columns)

%matplotlib inline
import matplotlib.pyplot as plt

n = 13
plt.figure(figsize=(10,n/2))
plt.title(f'Top {n} features')
importances.sort_values()[-n:].plot.barh();

# permutation importance
import eli5
from eli5.sklearn import PermutationImportance

permuter = PermutationImportance(
    a7,
    scoring='r2', 
    n_iter=5,
    random_state=2
)

permuter.fit(x_train_encoded, y_train);

import pandas as pd
feature_names = x_train.columns.tolist()

eli5.show_weights(
    permuter, 
    top=None,
    feature_names=feature_names
)

## *

> 건물면적과 대지권 면적이 가장 큰 영향을 미치는 변수임

In [ ]:
shap.initjs()
shap_values1 = explainer.shap_values(x_test_encoded.iloc[:300])
shap.summary_plot(shap_values1, x_test_encoded.iloc[:300], plot_type="violin")

## *

> * 건물면적과 대지권면적은 클수록 값이 커지는 경향.
> * 건축년도는 집값에 미치는 영향이 그렇게 크지 않아 보임.
> * 층정보는 낮을수록 집값이 뚜렷하게 감소.

In [ ]:
# 1. force plot
shap.initjs()
shap_values = explainer.shap_values(x_train_encoded.iloc[:1000])
shap.force_plot(explainer.expected_value, shap_values, x_train_encoded)

## *

> 다음과 같은 그래프로 예측해볼 수 있음.

In [ ]:
row = x_test_encoded.iloc[[1]]
y_test
a7.predict(row)
import shap
explainer = shap.TreeExplainer(a7)
shap_values = explainer.shap_values(row)
shap.initjs()
shap.force_plot(
    base_value=explainer.expected_value, 
    shap_values=shap_values,
    features=row
)

## *

> 테스트 데이터 하나로 예측해본 결과 법정동명과 자치구명이 집값을 높이는 영향을 주고, 나머지 변수는 낮추는 영향을 주는 것을 확인

# 결론

In [ ]:
dftr.columns